## Import and structure data

In [1]:
# Import libraries
import pandas as pd
import torchaudio
import os

c:\Users\hajko\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Import train data
df_train = pd.DataFrame(columns=["file_name", "accent", "gender", "audio"])
df_train.file_name = os.listdir("../data/train")
df_train.accent = [file_name[0] for file_name in df_train.file_name]
df_train.gender = [file_name[1] for file_name in df_train.file_name]
loaded_audio = []
# display(df_train)
sample_rates = set()
for file_name in df_train.file_name:
    tensor, sr = torchaudio.load(f"../data/train/{file_name}")
    loaded_audio.append(tensor)
    sample_rates.add(sr)

df_train.audio = loaded_audio
print(f"Sample rates in dataset: {sample_rates}")

Sample rates in dataset: {16000}


In [3]:
display(df_train)

,file_name,accent,gender,audio
0,1f_1018.wav,1,f,"[[tensor(-0.0002), tensor(-6.1035e-05), tensor..."
1,1f_1026.wav,1,f,"[[tensor(-0.0008), tensor(-0.0008), tensor(-0...."
2,1f_1031.wav,1,f,"[[tensor(-0.0002), tensor(-0.0002), tensor(-0...."
3,1f_1070.wav,1,f,"[[tensor(0.0022), tensor(0.0019), tensor(0.001..."
4,1f_1075.wav,1,f,"[[tensor(0.), tensor(3.0518e-05), tensor(0.), ..."
...,...,...,...,...
3161,5m_9719.wav,5,m,"[[tensor(0.0002), tensor(3.0518e-05), tensor(0..."
3162,5m_9742.wav,5,m,"[[tensor(0.0003), tensor(0.0002), tensor(0.000..."
3163,5m_9771.wav,5,m,"[[tensor(0.0003), tensor(0.0002), tensor(0.000..."
3164,5m_9778.wav,5,m,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0..."


In [12]:
# Display audio from DataFrame to make sure it loaded properly
import IPython.display as ipd
ipd.Audio(df_train.audio[0], rate=16000)

## Data analysis

In [48]:
#number of males and females in the data set
print(df_train['gender'].value_counts())
print(len(df_train))

gender
f    1639
m    1527
Name: count, dtype: int64
3166


In [57]:
# number of entried for each accents in the datset
accent_distribution = df_train['accent'].value_counts()
accent_distribution

accent
4    754
1    740
2    626
3    564
5    482
Name: count, dtype: int64

In [65]:
import matplotlib as plt

#plt.bar(accent_distribution)